# MatchCake Example

In this example, we will use the `matchcake` library to create a simple quantum circuit using the `NonInteractingFermionicDevice` and the `fRXX`, `fRYY`, `fRZZ`, and `fSWAP` operations. We will then use PennyLane to create a QNode and evaluate the expectation value of the circuit.

# Imports

We start by importing the necessary libraries.

In [ ]:
import matchcake as mc
import pennylane as qml
import numpy as np
from pennylane.ops.qubit.observables import BasisStateProjector

# Create the NonInteractingFermionicDevice

We start by creating a `NonInteractingFermionicDevice` with 4 wires. We also define the initial state as the zero state.

In [ ]:
nif_device = mc.NonInteractingFermionicDevice(wires=4)
initial_state = np.zeros(len(nif_device.wires), dtype=int)

# Define the circuit

We define a simple circuit that applies the `fRXX`, `fRYY`, `fRZZ`, and `fSWAP` operations to the initial state. We also include a `BasisStateProjector` to measure the expectation value of the circuit.

In [ ]:
def circuit(params, wires, initial_state=None):
    qml.BasisState(initial_state, wires=wires)
    for i, even_wire in enumerate(wires[:-1:2]):
        idx = list(wires).index(even_wire)
        curr_wires = [wires[idx], wires[idx + 1]]
        mc.operations.fRXX(params, wires=curr_wires)
        mc.operations.fRYY(params, wires=curr_wires)
        mc.operations.fRZZ(params, wires=curr_wires)
    for i, odd_wire in enumerate(wires[1:-1:2]):
        idx = list(wires).index(odd_wire)
        mc.operations.fSWAP(wires=[wires[idx], wires[idx + 1]])
    projector: BasisStateProjector = qml.Projector(initial_state, wires=wires)
    return qml.expval(projector)

# Create the QNode

We create a QNode using PennyLane and the `NonInteractingFermionicDevice` we defined earlier.

In [ ]:
nif_qnode = qml.QNode(circuit, nif_device)

# Draw the circuit

We can draw the circuit using PennyLane's `draw_mpl` function.

In [ ]:
qml.draw_mpl(nif_qnode)(np.array([0.1, 0.2]), wires=nif_device.wires, initial_state=initial_state)

# Evaluate the QNode

We can now evaluate the QNode using some random parameters.

In [ ]:
params = np.random.random(2)

In [ ]:
expval = nif_qnode(params, wires=nif_device.wires, initial_state=initial_state)

# Print the expectation value

We print the expectation value of the circuit.

In [ ]:
print(f"Expectation value: {expval}")